- Alura - LangChain: criando chatbots inteligentes com RAG

https://docs.langchain.com/oss/python/langchain/models#google-gemini

https://docs.pinecone.io/integrations/langchain

https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br

Procurei todas as bibliotecas usadas aqui:
https://docs.langchain.com/oss/python/langchain/knowledge-base#pinecone

# Arquitetura RAG com Gemini

## Configuração do Ambiente

In [ ]:
!pip install -q -U "langchain[google-genai]"

In [2]:
import os
from google.colab import userdata

In [3]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

## Prompting Tradicional vs RAG

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite', temperature=1)

In [6]:
# Exemplo 1: Prompting Tradicional (sem RAG)

pergunta = "Qual é a política de home office da nossa empresa?"

prompt_tradicional = llm.invoke(
    f'Responda a seguinte pergunta: {pergunta}'
)

In [7]:
prompt_tradicional.content

'Para te dar uma resposta precisa sobre a política de home office da sua empresa, preciso de mais informações. **Não tenho acesso direto aos dados internos da sua organização.**\n\nNo entanto, posso te guiar sobre como obter essa informação e quais são os pontos comuns que você pode esperar encontrar em uma política de home office.\n\n**Como descobrir a política de home office da sua empresa:**\n\n1.  **Consulte o RH (Recursos Humanos):** Esta é a fonte mais confiável. Entre em contato com o departamento de RH da sua empresa. Eles terão a política oficial documentada e poderão esclarecer todas as suas dúvidas.\n2.  **Verifique a Intranet ou Plataforma Interna:** Muitas empresas hospedam suas políticas e procedimentos em intranets, portais de funcionários, wikis internos ou outras plataformas colaborativas. Procure por seções como "Políticas", "Benefícios", "Trabalho Remoto" ou algo similar.\n3.  **Pergunte ao seu Gestor Direto:** Seu gestor provavelmente estará a par da política de hom

## Armazenamento Vetorial

- Os **Embeddings** criam um espaço multidimensional onde cada palavra ou documento é um ponto. Neste espaço, a proximidade geométrica representa a similaridade semântica.

- **Banco de dados vetorial**: são sistemas especializados em armazenar, indexar e buscar vetores de alta dimensão de forma eficiente. Diferente dos bancos de dados tradicionais, eles são otimizados para busca por similaridade, não por correspondência exata.

### Tipo de Índices

- **Flat (Força Bruta)**: Compara a consulta com todos os vetores, garantindo precisão perfeita, mas sendo muito lento para grandes volumes de dados
- **IVF (Inverted File Index)**: Agrupa vetores em clusters e busca apenas nos mais relevantes, sendo significativamente mais rápido que o Flat
- **HNSW (Hierarchical Navigable Small World)**: Constrói um grafo multicamadas para uma busca extremamente rápida e eficiente, ideal para sistemas em larga escala

In [8]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
!pip install -q faiss-cpu chromadb langchain-pinecone pinecone-client

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')

In [12]:
# Exemplo prático
embeddings.embed_query('Política de home office')

[-0.004343942739069462,
 0.025564059615135193,
 0.023318294435739517,
 -0.07726603001356125,
 0.0026561475824564695,
 0.00782716553658247,
 -0.0027659444604068995,
 -0.0017135508824139833,
 0.00522375525906682,
 -0.02445908822119236,
 -0.004932834301143885,
 -0.012425240129232407,
 -0.0020380273927003145,
 0.001390337711200118,
 0.125613272190094,
 -0.0007418602472171187,
 0.01239484827965498,
 0.01818224787712097,
 -0.006315963342785835,
 0.0021706297993659973,
 0.03742121160030365,
 0.008989612571895123,
 -0.003861862700432539,
 -0.021556153893470764,
 -0.02932857535779476,
 -0.015025743283331394,
 0.008016347885131836,
 0.012968025170266628,
 0.026312625035643578,
 -0.008344988338649273,
 -0.018941666930913925,
 0.03557060286402702,
 0.01356576569378376,
 0.01170012541115284,
 0.003974516876041889,
 0.025033172219991684,
 0.01595788635313511,
 0.0033185379579663277,
 -0.008064034394919872,
 0.020749138668179512,
 -0.01186464261263609,
 0.0025485926307737827,
 0.002566744340583682,
 

### Exemplo com Documentos

In [13]:
from langchain_core.documents import Document

In [40]:
documentos_empresa = [
    Document(
        page_content='Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.',
        metadata={'tipo': 'política', 'departamento': 'RH', 'ano': 2024, 'id_doc': 'doc001'}
    ),
    Document(
        page_content='Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.',
        metadata={'tipo': 'processo', 'departamento': 'Financeiro', 'ano': 2023, 'id_doc': 'doc002'}
    ),
    Document(
        page_content='Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as insturções para seu sistema operacional.',
        metadata={'tipo': 'tutorial', 'departamento': 'TI', 'ano': 2024, 'id_doc': 'doc003'}
    ),
    Document(
        page_content='Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.',
        metadata={'tipo': 'política', 'departamento': 'RH', 'ano': 2022, 'id_doc': 'doc004'}
    )
]


In [ ]:
!pip install -qU langchain-community

### FAISS

- Biblioteca para fazer busca de similariade rápida
- Duas estratégias:
    - **Índice Flat (Busca Exata)**:
        - Compara seu item de busca com *todos* os outros itens no banco de dados, um por um.
        - Precisão de 100%, porém muito lento
    - **Índice HNSW (Busca Aproximada)**:
        - Cria uma estrutura de dados otimizada (grafo) que permite pular comparaçoes desnecessárias.
        - A busca é guiada de forma inteligente para a área mais provável dos resultados.
        - Precisão não é 100%, busca boa e extremamente rápido. Ideal para tempo real

In [16]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [17]:
d = 768 # dimensão dos embeddings do Gemini, 768 valores por vetor
index_hnsw = faiss.IndexHNSWFlat(d, 32) # 32 vizinhos por nó

In [18]:
faiss_db = FAISS.from_documents(documentos_empresa, embeddings)

In [19]:
pergunta = 'Como peço minhas férias?'
resultado = faiss_db.similarity_search(pergunta, k=2) # Buscar os dois docs mais parecidos

In [20]:
resultado

[Document(id='54838e10-f9f7-4c5e-a2ca-011a9d56377a', metadata={'tipo': 'política', 'departamento': 'RH', 'ano': 2024, 'id_doc': 'doc001'}, page_content='Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.'),
 Document(id='256fac67-cc33-4fcb-94fa-06d28f12090a', metadata={'tipo': 'processo', 'departamento': 'Financeiro', 'ano': 2023, 'id_doc': 'doc002'}, page_content='Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.')]

### ChromaDB

- Bando de dados vetorial open-source
- Busca por similaridade
- Filtra por Metadados: permite associar informações adicionais (como datas, categorias, fontes) a cada vetor

In [21]:
from langchain_community.vectorstores import Chroma

In [22]:
chroma_db = Chroma.from_documents(
    documents=documentos_empresa,
    embedding=embeddings
)

In [23]:
resultados = chroma_db.similarity_search(pergunta, k=2)

In [24]:
resultados

[Document(metadata={'ano': 2024, 'id_doc': 'doc001', 'tipo': 'política', 'departamento': 'RH'}, page_content='Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.'),
 Document(metadata={'id_doc': 'doc002', 'tipo': 'processo', 'departamento': 'Financeiro', 'ano': 2023}, page_content='Processo de reembolso de despesas: Envie a nota fiscal pelo portal financeiro. O reembolso ocorre em até 5 dias úteis.')]

In [25]:
pergunta_rh = 'Quais são as regras da empresa?'

resultados_filtrados = chroma_db.similarity_search(
    pergunta_rh,
    k=2,
    filter={'$and': [{'departamento': 'RH'}, {'tipo': 'política'}]}
)

In [26]:
resultados_filtrados

[Document(metadata={'id_doc': 'doc004', 'tipo': 'política', 'ano': 2022, 'departamento': 'RH'}, page_content='Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.'),
 Document(metadata={'tipo': 'política', 'ano': 2024, 'departamento': 'RH', 'id_doc': 'doc001'}, page_content='Política de férias: Funcionários têm direito a 30 dias de férias após 12 meses. A solicitação deve ser feita com 30 dias de antecedência.')]

### Pinecone - Escalabilidade na Nuvem

- Banco de dados vetorial, oferecido como um serviço na nuvem (SaaS)
- **Alta escalabilidade**: Projetado para crescer junto com a aplicação, suportando bilhões de vetores e um alto volume de buscas
- **Disponibilidade e Confiança**: Garante que o bnaco de dados esteja sempre online, otimizado e seguro
- **Zero Manutenção**

In [45]:
from langchain_pinecone import Pinecone
from pinecone import Pinecone as PineconeCliente
from pinecone import ServerlessSpec

In [37]:
index_name = 'langchain-rag'
pinecone_client = PineconeCliente(api_key=os.environ['PINECONE_API_KEY'])
spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [43]:
# Verificando se o index existe. Se não, ele será criado
if index_name not in pinecone_client.list_indexes().names():
    pinecone_client.create_index(
        name=index_name,
        dimension=3072,
        metric='cosine',
        spec=spec
    )
    print(f'Índice {index_name} criado no Pinecone')

    pinecone_db = Pinecone.from_documents(
        documentos_empresa,
        embeddings,
        index_name=index_name
    )
    print(f'Documentos adicionados ao índice "{index_name}"')
else:
    print(f'Conectando ao índice existente "{index_name}"')
    # Se o índice já existe, apenas carregamos
    pinecone_db = Pinecone.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )

Índice langchain-rag criado no Pinecone
Documentos adicionados ao índice "langchain-rag"


In [44]:
if pinecone_db:
    pergunta_ti = 'Como configuro a VPN?'
    resultados_pinecone = pinecone_db.similarity_search(pergunta_ti, k=2)
    print(f'\nPergunta: {pergunta_ti}')
    print(f'Resultados: {resultados_pinecone}')

    # Busca por filtro
    resultados_pinecone_filtrados = pinecone_db.similarity_search(
        'Informações sobre regras',
        k=2,
        filter={'tipo': 'política'}
    )
    print(f'\nPergunta: "Informações sobre regras" com filtro para tipo="política"')
    print(f'Resultados: {resultados_pinecone_filtrados}')


Pergunta: Como configuro a VPN?
Resultados: [Document(id='f1a83446-2992-4415-91c5-01b687ac03e9', metadata={'ano': 2024.0, 'departamento': 'TI', 'id_doc': 'doc003', 'tipo': 'tutorial'}, page_content='Guia de TI: Para configurar a VPN, acesse vpn.nossaempresa.com e siga as insturções para seu sistema operacional.'), Document(id='36c8b281-bb6c-4947-a9d7-ed81f575a2a0', metadata={'ano': 2022.0, 'departamento': 'RH', 'id_doc': 'doc004', 'tipo': 'política'}, page_content='Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.')]

Pergunta: "Informações sobre regras" com filtro para tipo="política"

Resultados: [Document(id='36c8b281-bb6c-4947-a9d7-ed81f575a2a0', metadata={'ano': 2022.0, 'departamento': 'RH', 'id_doc': 'doc004', 'tipo': 'política'}, page_content='Código de Ética e Conduta: Valorizamos o respeito, a integridade e a colaboração. Casos de assédio não serão tolerados.'), Document(id='b5739dd4-b38f-4bac-a785-3ac65335

## Embeddings de Alta Performance

In [46]:
!pip install -q sentence-transformers

In [47]:
import time
import numpy

### Comparativo de Modelos: Gemini (API) vs Hugging Face (Local)

In [49]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import time

In [51]:
# Textos de exemplo para nosso teste
textos_teste = [
    'Qual é a política de férias da nossa empresa?',
    'Preciso de um relatório de despesas de viagem.',
    'Como configuro o acesso à rede privada virtual (VPN)?',
    'Onde encontro o código de conduta da organização?',
    'Quero entender o processo de avaliação de performance.'
]

In [52]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')

start_time = time.time()
embeddings_gemini = gemini_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f'Tempo de processamento: {end_time - start_time} segundos')
print(f' - Dimensões do vetor: {len(embeddings_gemini[0])}')

Tempo de processamento: 0.2089989185333252 segundos
 - Dimensões do vetor: 3072


In [ ]:
minilm_embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-l6-v2')

start_time = time.time()
embeddings_minilm = minilm_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f'Tempo de processamento: {end_time - start_time} segundos')
print(f' - Dimensões do vetor: {len(embeddings_minilm[0])}')

In [ ]:
bge_embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')

start_time = time.time()
embeddings_bge = bge_embeddings.embed_documents(textos_teste)
end_time = time.time()

print(f'Tempo de processamento: {end_time - start_time} segundos')
print(f' - Dimensões do vetor: {len(embeddings_bge[0])}')

### Análise de Qualidade Semântica

In [55]:
pergunta = 'Quero tirar uns dias de folga do trabalho.'

emb_pergunta_gemini = gemini_embeddings.embed_query(pergunta)
emb_pergunta_minilm = minilm_embeddings.embed_query(pergunta)
emb_pergunta_bge = bge_embeddings.embed_query(pergunta)

In [56]:
modelos = {
    'Gemini': (emb_pergunta_gemini, embeddings_gemini),
    'MiniLM': (emb_pergunta_minilm, embeddings_minilm),
    'BGE': (emb_pergunta_bge, embeddings_bge)
}

In [57]:
for nome, (emb_q, emb_docs) in modelos.items():
    similaridades = cosine_similarity([emb_q], emb_docs)[0]
    docs_e_similaridades = sorted(zip(textos_teste, similaridades),
                                  key=lambda x: x[1], reverse=True)
    print(f'--- Ranking para o modelo {nome} ----')
    for i, (doc, sim) in enumerate(docs_e_similaridades[:3], 1):
        print(f'{i}. (Score: {sim:.3f}) {doc}')
    print()

--- Ranking para o modelo Gemini ----
1. (Score: 0.831) Qual é a política de férias da nossa empresa?
2. (Score: 0.785) Preciso de um relatório de despesas de viagem.
3. (Score: 0.746) Quero entender o processo de avaliação de performance.

--- Ranking para o modelo MiniLM ----
1. (Score: 0.496) Quero entender o processo de avaliação de performance.
2. (Score: 0.469) Onde encontro o código de conduta da organização?
3. (Score: 0.465) Qual é a política de férias da nossa empresa?

--- Ranking para o modelo BGE ----
1. (Score: 0.646) Qual é a política de férias da nossa empresa?
2. (Score: 0.645) Preciso de um relatório de despesas de viagem.
3. (Score: 0.621) Onde encontro o código de conduta da organização?



### Caching de Embeddings: Economia e Velocidade

- Gerar embeddings, especialmente via API, tem custos de tempo e dinheiro. O cache armazena os embeddings já calculados para evitar retrabalhos.

In [58]:
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_classic.storage import LocalFileStore

In [63]:
store = LocalFileStore('./cache/')

embedder_principal = gemini_embeddings

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedder_principal,
    document_embedding_cache=store,
    namespace='gemini_cache'
)

In [64]:
textos_para_cache = ['Olá, mundo!', 'Testando o cache de embeddings', 'Olá, mundo!']

start_time = time.time()
embeddings_results_1 = cached_embeddings.embed_documents(textos_para_cache)
end_time = time.time()

print(f'Tempo de processamento: {end_time - start_time} segundos')
print(f' - Dimensões do vetor: {len(embeddings_results_1[0])}')

Tempo de processamento: 0.17679500579833984 segundos
 - Dimensões do vetor: 3072


### Batch Processing para Indexação em Larga Escala

In [66]:
documentos_grandes = [f'Este é um documento de teste número {i}.' for i in range(1000)]

bge_embedder = HuggingFaceEmbeddings(
    model_name='BAAI/bge-large-en-v1.5',
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

batch_sizes = [1, 32, 64, 128]

In [71]:
1000 // 128, 1000 % 128

(7, 104)

In [72]:
7 * (0.1 * 128) + (1000 % 128) * 0.1

100.00000000000001

In [73]:
for batch_size in batch_sizes:
    start_time = time.time()
    num_batches = len(documentos_grandes) // batch_size
    tempo_estimado = num_batches * (0.1 * batch_size) + (len(documentos_grandes) % batch_size) * 0.1
    tempo_real = bge_embedder.client.encode(documentos_grandes, batch_size=batch_size)
    end_time = time.time()

    print(f' - Batch Size: {batch_size:<4} -> Tempo: {end_time - start_time:.2f} segundos')

 - Batch Size: 1    -> Tempo: 431.15 segundos
 - Batch Size: 32   -> Tempo: 216.42 segundos
 - Batch Size: 64   -> Tempo: 207.40 segundos
 - Batch Size: 128  -> Tempo: 210.77 segundos


## Pipeline para Dados Complexos

- Pipelines são essenciais para RAG
- **Qualidade de Contexto**: A forma como os dados são processados e divididos (chunking) afeta diretamente o contexto que o LLM recebe e, portanto, a qualidade da resposta
- **Diversidade de Fontes**: Sistemas de RAG em produção se alimentam de múltiplas fontes: PDFs, banco de dados, APIs, etc
- **Metadados**: São a chave para buscas filtradas, rastreabilidade e controle de acesso, tornando o RAG muito mais poderoso

In [ ]:
!pip install -q "unstructured[pdf]" langchain-unstructured duckdb

In [75]:
import duckdb
import pandas as pd
from datetime import datetime

### Processando PDFs Complexos com Unstructured

In [78]:
from langchain_unstructured import UnstructuredLoader

In [79]:
loader = UnstructuredLoader('relatorio_vendas_RAG.pdf')
docs_unstructured = loader.load()
print(f'Total de elementos extraídos: {len(docs_unstructured)}\n')

for doc in docs_unstructured:
    print(f'---- TIPO DE ELEMENTO: {doc.metadata.get("category")} ----')
    print(doc.page_content)
    print('\n')

Total de elementos extraídos: 23

---- TIPO DE ELEMENTO: NarrativeText ----
Relatório Trimestral de Vendas - Q1 2024 Este relatório apresenta uma análise detalhada das vendas no primeiro trimestre de 2024. A performance geral foi positiva, com crescimento em todas as categorias de produtos. A seguir, uma tabela com os resultados por produto.


---- TIPO DE ELEMENTO: Title ----
ID Produto


---- TIPO DE ELEMENTO: Title ----
Nome do Produto


---- TIPO DE ELEMENTO: Title ----
Categoria Unidades Vendidas Receita (R$)


---- TIPO DE ELEMENTO: Title ----
PROD-001


---- TIPO DE ELEMENTO: Title ----
Laptop Pro X


---- TIPO DE ELEMENTO: Title ----
Eletrônicos


---- TIPO DE ELEMENTO: UncategorizedText ----
1500


---- TIPO DE ELEMENTO: UncategorizedText ----
7.500.000


---- TIPO DE ELEMENTO: Title ----
PROD-002 Cadeira Ergonômica Mobiliário


---- TIPO DE ELEMENTO: UncategorizedText ----
2500


---- TIPO DE ELEMENTO: UncategorizedText ----
1.250.000


---- TIPO DE ELEMENTO: Title ----
PROD-

### Adicionando Metadados Estratégicos na Carga

In [81]:
docs_com_metadados = []

for doc in docs_unstructured:
    novos_metadados = doc.metadata.copy()

    novos_metadados['source'] = 'relatorio_vendas_RAG.pdf'
    novos_metadados['ingestion_date'] = datetime.now().strftime('%Y-%m-%d')
    novos_metadados['data_owner'] = 'Departamento de Vendas'

    docs_com_metadados.append(
        Document(page_content=doc.page_content, metadata=novos_metadados)
    )

print(f'Total de documentos com metadados: {len(docs_com_metadados)}\n')
print(docs_com_metadados[-1])

Total de documentos com metadados: 23

page_content='A categoria de Eletrônicos continua a ser a mais lucrativa. A estratégia para o Q2 será focar em marketing para a Cadeira Ergonômica, que possui alto volume de vendas mas menor receita.' metadata={'source': 'relatorio_vendas_RAG.pdf', 'coordinates': {'points': ((78.0, 291.07), (78.0, 313.07), (502.6299999999997, 313.07), (502.6299999999997, 291.07)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'filename': 'relatorio_vendas_RAG.pdf', 'last_modified': '2025-11-02T21:26:06', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'parent_id': 'da148852c51bebd997508d23d1abfd3f', 'category': 'NarrativeText', 'element_id': 'e536eafcd4153232bb2be9910bc4063a', 'ingestion_date': '2025-11-02', 'data_owner': 'Departamento de Vendas'}


### Chunking Inteligente com RecursiveCharacterTextSplittler

In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [83]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

chunks = text_splitter.split_documents(docs_com_metadados)

print(f'Número de documentos original: {len(docs_com_metadados)}')
print(f'Número de chunks gerados: {len(chunks)}\n')

print(chunks[2])

Número de documentos original: 23
Número de chunks gerados: 23

page_content='Nome do Produto' metadata={'source': 'relatorio_vendas_RAG.pdf', 'coordinates': {'points': ((177.86, 171.07000000000005), (177.86, 181.07000000000005), (261.75, 181.07000000000005), (261.75, 171.07000000000005)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'filename': 'relatorio_vendas_RAG.pdf', 'last_modified': '2025-11-02T21:26:06', 'page_number': 1, 'languages': ['por'], 'filetype': 'application/pdf', 'category': 'Title', 'element_id': '9f9bc7bd67badf9acda4c3666a69b233', 'ingestion_date': '2025-11-02', 'data_owner': 'Departamento de Vendas'}


### Ingestão de Dados de um Banco SQL com DuckDB

- Cenário: Temos um banco de dados de "Produtos" e queremos que nosso RAG possa responder perguntas sobre eles

In [84]:
# Conectar ao DuckDB (ele cria o arquivo se não existir)
con = duckdb.connect(database=':memory:', read_only=False)

# Cria uma tabela de produtos
con.execute("""
    CREATE TABLE produtos (
        id INTEGER,
        nome VARCHAR,
        categoria VARCHAR,
        preco FLOAT,
        estoque INTEGER,
        descricao VARCHAR
    );
""")

# Inserir dados de exemplo
produtos_df = pd.DataFrame({
    'id': [101, 102, 103, 104],
    'nome': ['Laptop Gamer Z', 'Mouse Óptico Fast', 'Teclado Mecânico Pro', 'Monitor Curvo 34"'],
    'categoria': ['Eletrônicos', 'Acessórios', 'Acessórios', 'Eletrônicos'],
    'preco': [9500.00, 250.00, 800.00, 3200.00],
    'estoque': [15, 120, 60, 25],
    'descricao': [
        'Laptop de alta performance com placa de vídeo dedicada e 32GB RAM.',
        'Mouse com 16.000 DPI e design ergonômico para longas sessões',
        'Teclado com switches mecânicos, RGB e layout ABNT2',
        'Monitor ultrawide com alta taxa de atualização e cores vibrantes'
    ]
})

con.register('produtos_df', produtos_df)
con.execute('INSERT INTO produtos SELECT * FROM produtos_df')

print('Tabela "produtos" criada e dados inseridos com sucesso.')

# Verificar os dados
print(con.execute('SELECT * FROM produtos;').fetchdf())

Tabela "produtos" criada e dados inseridos com sucesso.
    id  ...                                          descricao
0  101  ...  Laptop de alta performance com placa de vídeo ...
1  102  ...  Mouse com 16.000 DPI e design ergonômico para ...
2  103  ...  Teclado com switches mecânicos, RGB e layout A...
3  104  ...  Monitor ultrawide com alta taxa de atualização...

[4 rows x 6 columns]


#### Transformando Linhas SQL em Documentos

In [85]:
# Query para selecionar os dados
df_produtos = con.execute("SELECT * FROM produtos;").fetchdf()

# Lista para armazenar os documentos
docs_sql = []

for _, row in df_produtos.iterrows():
    # Criar um texto descritivo a partir da linha
    page_content = f"""Produto: {row['nome']}. Categoria: {row['categoria']}.
                Preço: R${row['preco']:.2f}. Em estoque: {row['estoque']} unidades.
                Descrição: {row['descricao']}"""

    # Criar metadados estratégicos
    metadata = {
        'source': 'tabela_produtos_duckdb',
        'produto_id': row['id'],
        'categoria': row['categoria'],
        'preco': row['preco'],
        'ingestion_date': datetime.now().strftime('%Y-%m-%d')
    }

    docs_sql.append(
        Document(page_content=page_content, metadata=metadata)
    )

# Fechar a conexão com o banco
con.close()

print(f"Total de documentos gerados a partir do SQL: {len(docs_sql)}\n")
print("Exemplo de documento gerado a partir de uma linha do banco de dados:")
print(docs_sql[0])

Total de documentos gerados a partir do SQL: 4

Exemplo de documento gerado a partir de uma linha do banco de dados:
page_content='Produto: Laptop Gamer Z. Categoria: Eletrônicos. 
                Preço: R$9500.00. Em estoque: 15 unidades. 
                Descrição: Laptop de alta performance com placa de vídeo dedicada e 32GB RAM.' metadata={'source': 'tabela_produtos_duckdb', 'produto_id': 101, 'categoria': 'Eletrônicos', 'preco': 9500.0, 'ingestion_date': '2025-11-02'}


### Unindo os Pipelines e Enviando para o Vector Store

In [88]:
from langchain_community.vectorstores.utils import filter_complex_metadata

In [89]:
documentos_finais = chunks + docs_sql

print(f'Total de documentos a serem indexados: {len(documentos_finais)}')

documentos_filtrados = filter_complex_metadata(documentos_finais)

print(f'Total de documentos filtrados: {len(documentos_filtrados)}')

embeddings = gemini_embeddings

vector_store = Chroma.from_documents(
    documents=documentos_filtrados,
    embedding=embeddings
)

Total de documentos a serem indexados: 27
Total de documentos filtrados: 27


### Testando o Resultado Final

In [92]:
pergunta_pdf = 'Qual foi a receita com laptops?'

resultados_pdf = vector_store.similarity_search(pergunta_pdf, k=2)

print(f'Pergunta: {pergunta_pdf}\n')

for doc in resultados_pdf:
    print(f'- Similaridade: {doc.page_content}')
    print(f' (Fonte: {doc.metadata.get("source")}, Categoria: {doc.metadata.get("category")})')

print('-' * 20)

# Pergunta sobre o Banco de Dados

pergunta_sql = 'Me fale sobre o teclado mecânico'
resultado_sql = vector_store.similarity_search(pergunta_sql, k=2)

print(f'Pergunta: {pergunta_sql}\n')
for doc in resultado_sql:
    print(f'- Similaridade: {doc.page_content}')
    print(f' (Fonte: {doc.metadata.get("source")}, Categoria: {doc.metadata.get("category")})')

Pergunta: Qual foi a receita com laptops?

- Similaridade: Eletrônicos
 (Fonte: relatorio_vendas_RAG.pdf, Categoria: Title)
- Similaridade: Eletrônicos
 (Fonte: relatorio_vendas_RAG.pdf, Categoria: Title)
--------------------
Pergunta: Me fale sobre o teclado mecânico

- Similaridade: Eletrônicos
 (Fonte: relatorio_vendas_RAG.pdf, Categoria: Title)
- Similaridade: Eletrônicos
 (Fonte: relatorio_vendas_RAG.pdf, Categoria: Title)
